This notebook is based on https://github.com/vahidk/EffectiveTensorflow.  
___

### Doing computation in tensorflow
Tensorflow give us tensor which like a placeholder for our variable.
A tf.Tensor object represents a partially defined computation that will eventually produce a value. TensorFlow programs work by first building a graph of tf.Tensor objects.  
In order to compute the value of the tensor we need to create a session and evaluate it using Session.run() method.

In [2]:
import tensorflow as tf

x = tf.random_normal([10, 10])
y = tf.random_normal([10, 10])
z = tf.matmul(x, y)


sess = tf.Session()
z_val = sess.run(z)
print("this is z ",z)
print(z_val)

this is z  Tensor("MatMul_1:0", shape=(10, 10), dtype=float32)
[[  0.98684156   2.863435    -1.6667035   -0.8425131   -1.7591304
    0.2621978   -0.4838299    3.8139358    1.0740364   -0.4735332 ]
 [  0.1721021    0.39912608  -1.2777555   -0.7923951   -0.9775923
   -0.16850671  -4.5594683   -1.2539387   -0.04540014   0.36295214]
 [  3.0229654   -1.1301382    5.3861213   -3.376586     2.1859884
   -3.6515145   -0.59590495  -2.1741357    2.416408     2.274809  ]
 [ -5.1114025   -6.185856     6.9744883   -2.6803067   -0.9619111
   -0.04638492   4.5675793    2.4024372    2.6572094   -0.8861291 ]
 [  1.7131717   -0.06711298   6.6898293   -5.775112    -1.9994012
    3.7297204   -4.26378     -1.8154532  -13.651993    -2.3754747 ]
 [ -1.0223211    6.627446    -1.2646064    0.48809242   1.3152277
   -0.2615651   -0.6642335    2.4925654    3.3332803    2.2760768 ]
 [ -0.45090076   1.2121685    5.0687294   -2.3027318   -3.1910555
    3.8441775   -1.0027664   -1.3575561   -4.9749346    2.625601  ]

### Symbolic computation
Symbolic computation is a very different way of programming.

Classical programming defines variables that hold values, and operations to modify their values.

In symbolic programming, it’s more about building a graph of operations, that will be compiled later for execution. Such an architecture enables the code to be compiled and executed indifferently on CPU or GPU for example. Symbols are an abstraction that does not require to know where it will be executed.

let's say.  
- $f(x) = 5x^2 + 3$  
Parametric function  
- $g(x, w) = w_{0} x^2 + w_{1} x + w_{2}$  
our goal is then to find the latent parameters w such that   
- $g(x, w) ≈ f(x)$  
his can be done by minimizing the following loss function:  
- $L(w) = ∑ (f(x) - g(x, w))^2$   
   
To solve this problem we can use stocasthic gradient decent
which is simply update the value of w by moving it on it's gradient by
computing derivative of $L(w)$ and move it to opposite direction (minimizing the loss). 

In [14]:
import numpy as np
import tensorflow as tf

# Placeholder are used to feed value from python to tensorflow ops. We define two placeholders
# One for input feature x and one for output feature y
tf.reset_default_graph()
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# Assuming we know that the desired function is a polynomial 2nd degree, 
# we allocate a vector of size 3 to hold the coefficient. The variable will 
# be automatically initiated with random noise
## tf.get_variable just set variable which a name, so it can be called on different variable but with 
## same variable w
w = tf.get_variable("w", shape=[3,1]) #, initializer = tf.random_normal_initializer())

# we define yhat which is our estimate of y
## tf.square compute x*x
## tf.ones_like fill x with 1
## tf.st [w,w,w] * [x^2, x , 1] **I still don't understand why they put squeeze in here**
f = tf.stack([tf.square(x), x, tf.ones_like(x)], 1)
yhat = tf.squeeze(tf.matmul(f, w), 1)

# The loss is L2 distance from estimate y and predicted value y
# we also added shrinkage item (L1 norm)
loss = tf.nn.l2_loss(yhat - y) + 0.1 * tf.nn.l2_loss(w)

# We use the Adam optimizer with learning rate set to 0.1 to minizmise the loss.
train_op = tf.train.AdamOptimizer(0.1).minimize(loss)

def generate_data():
    x_val = np.random.uniform(-10.0, 10.0, size=100)
    y_val = 5 * np.square(x_val) + 3
    return x_val, y_val 

sess = tf.Session()
# Since we are using varibles we first need to initialize them.
sess.run(tf.global_variables_initializer())
for _ in range(1000):
    x_val, y_val = generate_data()
    _, loss_val = sess.run([train_op, loss],{x: x_val, y: y_val})
#     print(loss_val)
print(sess.run([w]))


[array([[4.9946995e+00],
       [5.1860115e-04],
       [3.3083560e+00]], dtype=float32)]


If you look at the result above is the predicted value of w
___
### Understanding static and dynamic shapes
#### Static shapes
Static types is defined shapes.
e.g array with element size 255
#### Dynamic shapes
If you want to feed the neural network with a number of batch
that you can modify, then just define the first dimension as zero or -1. If you look the code below, the first dimension will be deterimend dynamically during Session.run()

In [17]:
a = tf.placeholder(tf.float32, [None,128])

To get the dynamic shape of tensor

In [18]:
tf.shape(a)

<tf.Tensor 'Shape:0' shape=(2,) dtype=int32>

The static shape of tensor can be set with Tensor.set_shape() method:

In [ ]:
a.set_shape([32, 128])
a.set_shape([None,, 128]) # first dimension of a will bedetermined dynamically

You can also reshape a give tensor dynamically using tf.reshape function

In [19]:
a = tf.reshape(a, [32,128])

In [23]:
tf.unstack(tf.shape(a))

[<tf.Tensor 'unstack:0' shape=() dtype=int32>,
 <tf.Tensor 'unstack:1' shape=() dtype=int32>]

In [20]:
def get_shape(tensor):
   static_shape = tensor.shape.as_list()
   dynamic_shape = tf.unstack(tf.shape(tensor))
   dims = [s[1] if s[0] is None else s[0]
             for s in zip(static_shape, dynamic_shape)]
   return dims

In [ ]:
b = tf.placeholder(tf.float32, [None, 10, 32])
shape = get_shape(b)
b = tf.reshape(b, [shape[0], shape[1] * shape[2]])